<a href="https://colab.research.google.com/github/petitbato/Advanced_AI_Lab/blob/main/Advanced_AI_labs_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXAM Predicting German Credit Risk




## 1) Instructions and model training

### Instructions

For this advanced AI labs exam you are asked to complete this notebook with **code, graphs, explainations and analysis** depending on what is requested.

In this first section we introduce a dataset around credit risk, we preprocess its data and we train an xgboost model on it. Read this section attentively first to fully understand the context and to respond correctly afterwards.

The second, third and fourth sections contain instructions on how to analyze an improve the bias of the model, how to explain some decisions and how to attack the model.

To implement these instructions you are **free to use any ressource** you want (slides, previous colabs, only resources, AI chatbots, ...). The only rule is to **not communicate between each other or with someone outside the classroom**. You can, on demand, get out of the classroom for a short break or to go to the bathroom but only if no other student is already outside. You have a total of **3h** to complete everything.

At the end of the time you are asked to submit **a file .ipynb**  on claco with the name Lastname_Firstname_studentCode.ipynb. Make sure that the prints and graphs are displayed correctly in the code cells output (I won't run you code)and that the text cells are filled.



### Context

[The original dataset](https://colab.research.google.com/drive/1aM-dd5scoPEtLWBsFn8GoVg6G-6YBa-Z#scrollTo=8K_v6sDrOVSH&line=13&uniqifier=1) contains 1000 entries with 20 categorial/symbolic attributes prepared by Prof. Hofmann. In this dataset, each entry represents a person who takes a credit by a bank. Each person is classified as good or bad credit risks according to the set of attributes. It is almost impossible to understand the original dataset due to its complicated system of categories and symbols. Thus, a small Python script was written to convert it into a readable CSV file that we pull from [kaggle](https://www.kaggle.com/code/heidarmirhajisadati/credit-risk-prediction-using-german-credit-data/notebook). Several columns are simply ignored, because of their probable uselessness. The selected attributes are:

 *   Age (numeric)
 *   Sex (text: male, female)
 *   Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
 *   Housing (text: own, rent, or free)
 *   Saving accounts (text - little, moderate, quite rich, rich)
 *   Checking account (text - little, moderate, quite rich, rich)
 *   Credit amount (numeric, in DM)
 *   Duration (numeric, in month)
 *   Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, * repairs, education, business, vacation/others)

### Dataset Loading and pre-processing
We will start by loading the dataset from kaggle and preprocess the features to obtain numerical inputs.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
import os
import tensorflow as tf

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

seed = 11
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# URL of the CSV file on GitHub
url = "https://raw.githubusercontent.com/ziadasal/Credit-Risk-Assessment/main/german_credit_data.csv"

# Read the CSV file directly into a pandas DataFrame
df = pd.read_csv(url)

# Display the first few rows of the dataset
df.head(10)

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
5,5,35,male,1,free,NaN,NaN,9055,36,education,good
6,6,53,male,2,own,quite rich,NaN,2835,24,furniture/equipment,good
7,7,35,male,3,rent,little,moderate,6948,36,car,good
8,8,61,male,1,own,rich,NaN,3059,12,radio/TV,good
9,9,28,male,3,own,little,moderate,5234,30,car,bad


We will now process the dataset by encoding categorical features and handling missing values.

In [ ]:
# Drop purpose and unnamed data
df.drop('Purpose', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
# Handle missing values by filling with a placeholder
df.fillna({'Saving accounts': 'unknown', 'Checking account': 'unknown'}, inplace=True)

# Create a dictionary to store the order for each feature
feature_order = {
    'Sex': ['male', 'female'],
    'Housing': ['own', 'rent', 'free'],
    'Saving accounts': ['unknown', 'little','moderate','quite rich', 'rich'],
    'Checking account': ['unknown', 'little', 'moderate','quite rich',  'rich'],
    'Risk': ['bad', 'good']
}

# Create an OrdinalEncoder with the specified categories
categorical_features = list(feature_order.keys())
encoder = OrdinalEncoder(categories=[feature_order[feature] for feature in categorical_features])

# Fit and transform the data for selected columns
df[categorical_features] = encoder.fit_transform(df[categorical_features])

# Display the cleaned and encoded dataset
df.head()



,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Risk
0,67,0.0,2,0.0,0.0,1.0,1169,6,1.0
1,22,1.0,2,0.0,1.0,2.0,5951,48,0.0
2,49,0.0,1,0.0,1.0,0.0,2096,12,1.0
3,45,0.0,2,2.0,1.0,1.0,7882,42,1.0
4,53,0.0,2,2.0,1.0,1.0,4870,24,0.0


Separate the target y from the input data x and separate the training and test set.

In [ ]:
# Assuming 'df' is your DataFrame and 'Risk' is the target column
X = df.drop('Risk', axis=1)  # Features (all columns except 'Risk')
y = df['Risk']  # Target variable



In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Age               1000 non-null   int64  
 1   Sex               1000 non-null   float64
 2   Job               1000 non-null   int64  
 3   Housing           1000 non-null   float64
 4   Saving accounts   1000 non-null   float64
 5   Checking account  1000 non-null   float64
 6   Credit amount     1000 non-null   int64  
 7   Duration          1000 non-null   int64  
 8   Risk              1000 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 70.4 KB
None


### Model training

In [ ]:
def create_model(X_train, y_train):
    # Initialize the GradientBoostingClassifier with desired parameters
    model = GradientBoostingClassifier(
        n_estimators=80,  # Number of boosting stages (trees)
        learning_rate=0.1,  # Step size shrinkage used in update to prevents overfitting
        max_depth=3,  # Maximum depth of the individual trees
        random_state=11,  # Random seed for reproducibility
    )

    # Train the model
    model.fit(X_train, y_train)
    return model

model = create_model(X_train, y_train)
accuracy = accuracy_score(y_train, model.predict(X_train))
print(f"Training accuracy: {accuracy:.3f}")
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"Test accuracy: {accuracy:.3f}")

Test accuracy: 0.866
Test accuracy: 0.710


## 2) Biais

For this section when asked to compute the prevalence and accuracy, compute it **on the test set**.

### Find and measure bias

For the risk model find two features that could represent a bias with a negative societal impact, explain why they represent a risk of negative societal impact and measure the prevalence  ($\frac{TP+FN}{TP+FN+FP+TN}$) difference between two groups for both of these features to check if one has an easiest access to credit than the other.

In [ ]:
# Create a function that computes the prevalence based on the model the input_data,
#  the target and the threshold with a default value of 0.5

#Separate the dataset in two groups based on Feature 1

#Compute the prevalence for the two groups

#Separate the dataset in two groups based on Feature 2

#Compute the prevalence for the two groups

MODIFY THE TEXT HERE TO ANALYZE THE PREVALENCE DIFFERENCE AND EXPLAIN WHICH GROUP HAS AN ADVANTAGE.






### Pre-processing bias management

Apply a pre-processing technique of your choice (droping the sensitive feature, resampling, disparate impact remover, ...) **for one of the bias senstive feature** to modify the training data and create a new model using the function create_model(). Then compute the prevalence difference between the two groups and compare it to the model without the pre-processing. Check if this had an impact on the accuracy of your model.

In [ ]:
#create a new dataset by applying a bias pre-processing technique

#create the model based on this new dataset

#Separate the dataset in two groups based on the feature

#Compute the prevalence for the two groups (reuse the function of the previous function) and there difference

#Compute the average accuracy for this new model



MODIFY THE TEXT HERE TO ANALYZE THE PREVALENCE DIFFERENCE  BETWEEN THE TWO GROUPS BEFORE AND AFTER THE PREPROCESSING TECHNIQUE. CHECK HOW THE PREPROCESSING IMPACTED THE ACCURACY AND ANALYZE WHY.

### Post-processing bias management

For the same feature as for the pre-processing apply a post-processing technique to fight the bias. The technique that you are asked to use is the use of two different threshold for the two groups. To do so, find two thresholds that give the same prevalence for the two groups of 0.8. Check if this had an impact on the general accuracy. Use the model by default and not the one created in the bias pre-processing section. Plot the graph of the prevalence against the threshold to help you.

In [ ]:
#Code to obtain the two thresholds

MODIFY THE TEXT HERE TO ANALYZE THE PREVALENCE DIFFERENCE BETWEEN THE TWO GROUPS BEFORE AND AFTER THE POSTPROCESSING TECHNIQUE. CHECK HOW THE POSTPROCESSING IMPACTED THE AVERAGE ACCURACY AND ANALYZE WHY.

## 3) Explainability


For this section use the default model trained in the first instructions section.

### Feature permutation importance

Compute the feature permutation importance for each feature and analyze the results.

In [ ]:
#code for the feature permutation importance


ANALYZE HERE THE RESULTS (IT CAN BE SHORT)

###Partial dependance plot

Show the partial dependance plot of the model postive (good) credit risk rate for the three continuous features (age, credit_amount and duration) and analyze the results.

In [ ]:
#code for the partial dependance plot


ANALYZE OF THE PARTIAL DEPENDANCE PLOT

### Integrated Gradient

Louise is 23, she is a woman (value 1) and she has a master (and is hence highly skilled). For the moment she rents an appartement and she wants to buy a new couch. For this she would need to take loan of 5000 on 12 months. She has a low checking account (value 1) and a low saving account (value 1). When she asks for the loan it's declined and she doesn't understand why, hence, she asks her friend john (characteristics in the code below) to apply for the same loan and he gets it. She wants to understand why john got the loan and she didn't and she heared of the integrated gradient technique. She asks you to implement it and explain here what features made de difference in the model decision.

In [ ]:
# Create a dictionary with Louise's information
louise_data = {
    'Age': [20],
    'Sex': [1],
    'Job': [3],  # Highly skilled (master)
    'Housing': [1],
    'Saving accounts': [1],
    'Checking account': [1],
    'Credit amount': [5000],
    'Duration': [12]
}

john_data = {
   'Age': [45],
    'Sex': [0],
    'Job': [2],
    'Housing': [0],
    'Saving accounts': [3],
    'Checking account': [3],
    'Credit amount': [5000],
    'Duration': [12]
}

# Create a DataFrame from the dictionary
louise_df = pd.DataFrame(louise_data)
john_df = pd.DataFrame(john_data)

# Assuming 'model' is your trained TensorFlow model
prediction_probability_louise = model.predict_proba(louise_df)[0][1]  # Get the probability
prediction_probability_john = model.predict_proba(john_df)[0][1]  # Get the probability

print(f"The probability of Louise having a good credit risk is: {prediction_probability_louise * 100:.2f}%")
print(f"The probability of John having a good credit risk is: {prediction_probability_john * 100:.2f}%")

The probability of Louise having a good credit risk is: 44.51%
The probability of John having a good credit risk is: 63.72%


In [ ]:
#Implementation of the integrated gradient technique

EXPLAINATION FOR LOUISE

## 4) Security

### Black box attack

Louise whose situation was explained in the Integrated Gradient section wants to reapply for a loan. She is ready to lie about her age, to change the duration of the loan to maximum 2 months and to change the amount of the loan (if the loan stays between 4900 and 5100). She doesn't have access to the model inside weights and wants you to make a black box attack that would allow here to get a prediction above 0.5 for the risk but by lying and by changing her loan conditions as little as possible. Propose such a solution and propose a smart algorithm to obtain it. Pay attention to the scale (month, year vs euro)

In [ ]:
#Code of the black box attack

WRITE HERE WHAT SHE NEEDS TO CHANGE IN ORDER TO GET THE LOAN

### Data poisoning



Louise whose situation was explained in the Integrated Gradient section got access to the training data, she decides to poison it so that when she applies with her real information and wanted loan conditions she gets a positive response.

Add data to the training dataset so that after retraining the model Louise's application gets approved, check the impact it has on the test set accuracy and find a method that impacts the test set accuracy as little as possible so that nobody notices the poisoning.

*Advice: create a new variable to old the poisoined dataset so that when you go back to other exercices you don't use the wrong data.*

In [ ]:
#Code of the data poisoining

DESCRIBE YOUR DATA POISONING STRATEGY AND HOW YOU LIMITED THE IMPACT ON THE TEST SET ACCURACY